### Game Data

In [1]:
import numpy as np

In [2]:
!pip install nba_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import franchiseplayers
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

In [4]:
team_dict = teams.get_teams()
team_ids = []
for val in team_dict:
    team_ids.append(val['id'])
team_games = leaguegamefinder.LeagueGameFinder(player_or_team_abbreviation='T',season_type_nullable='Regular Season',season_nullable='2022-23').get_data_frames()[0]
nba_team_games = team_games[team_games.TEAM_ID.isin(team_ids)]
nba_team_games = nba_team_games.reset_index()
del nba_team_games['index']
nba_team_games['GAME_DATE'] = pd.to_datetime(nba_team_games['GAME_DATE'],format='%Y-%m-%d')

In [5]:
nba_team_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,-22.0
1,22022,1610612741,CHI,Chicago Bulls,0022201223,2023-04-09,CHI vs. DET,W,240,103,40,95,0.421,6,30,0.200,17,22,0.773,11,37,48,21,15,7,3,16,22.0
2,22022,1610612761,TOR,Toronto Raptors,0022201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0
3,22022,1610612737,ATL,Atlanta Hawks,0022201216,2023-04-09,ATL @ BOS,L,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,-6.0
4,22022,1610612745,HOU,Houston Rockets,0022201222,2023-04-09,HOU @ WAS,W,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22022,1610612765,DET,Detroit Pistons,0022200003,2022-10-19,DET vs. ORL,W,239,113,40,94,0.426,14,38,0.368,19,24,0.792,12,29,41,31,11,4,12,21,4.0
2456,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0
2457,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,40,94,0.426,10,40,0.250,19,25,0.760,9,39,48,23,12,4,21,18,-14.0
2458,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,45,99,0.455,16,45,0.356,17,23,0.739,11,37,48,31,11,4,18,23,14.0


In [6]:
nba_team_games[nba_team_games['FG_PCT']>=0.60]['TEAM_NAME'].value_counts()

Minnesota Timberwolves    6
Brooklyn Nets             6
Denver Nuggets            5
LA Clippers               4
Cleveland Cavaliers       3
Washington Wizards        3
Philadelphia 76ers        3
Chicago Bulls             3
Memphis Grizzlies         2
Golden State Warriors     1
Portland Trail Blazers    1
Los Angeles Lakers        1
Detroit Pistons           1
Houston Rockets           1
Charlotte Hornets         1
Toronto Raptors           1
New York Knicks           1
Dallas Mavericks          1
Milwaukee Bucks           1
Sacramento Kings          1
Oklahoma City Thunder     1
Name: TEAM_NAME, dtype: int64

In [7]:
1982/2

991.0

In [8]:
# Webscraping
resp = requests.get('https://www.nba.com/stats/teams/advanced')

html_code = resp.text

soup = BeautifulSoup(html_code,'lxml')

In [9]:
table = soup.find('table',attrs={'class':'Crom_table__p1iZz'})
#headers = table.findAll('th')
#headerlist = [h.text.strip() for h in headers[1:]]

In [10]:
advanced_df = pd.read_excel('/content/drive/MyDrive/SJ+PSG /Forecast/NBA/Teams Advanced _ Stats _ NBA.com.xlsx')
advanced_df

,TEAM,GP,W,L,MIN,OffRtg,DefRtg,NetRtg,AST%,AST/TO,AST Ratio,OREB%,DREB%,REB%,TOV%,eFG%,TS%,PACE,PIE,POSS
0,Milwaukee Bucks,67,48,19,3246,113.4,109.6,3.8,59.7,1.70,17.7,28.7,74.4,52.1,14.5,54.8,57.8,101.22,52.4,6848
1,Boston Celtics,68,47,21,3324,116.7,111.1,5.6,62.8,1.96,19.0,25.0,74.9,50.5,13.3,56.4,60.0,99.28,53.0,6870
2,Denver Nuggets,67,46,21,3231,117.3,113.3,4.0,66.7,1.98,20.7,28.4,73.3,51.5,14.8,57.8,60.7,99.18,52.9,6676
3,Philadelphia 76ers,66,44,22,3193,116.7,112.6,4.2,61.8,1.85,18.6,25.4,72.0,49.3,13.7,56.1,60.7,97.83,52.3,6507
4,Cleveland Cavaliers,69,42,27,3352,115.1,109.6,5.5,60.4,1.83,18.6,27.5,72.5,50.3,14.1,55.6,59.0,96.14,53.4,6713
5,Memphis Grizzlies,66,40,26,3178,113.7,109.7,4.0,58.8,1.86,17.8,30.8,70.9,50.8,13.4,53.1,56.2,101.63,51.9,6722
6,Sacramento Kings,66,40,26,3193,118.7,115.8,2.9,62.0,1.93,19.2,25.9,73.3,49.9,13.8,57.7,61.4,101.37,51.4,6741
7,New York Knicks,69,39,30,3367,116.0,113.5,2.5,53.7,1.73,16.2,32.0,72.0,51.9,13.0,53.3,57.0,97.47,50.6,6841
8,Brooklyn Nets,67,38,29,3226,114.3,113.2,1.1,60.5,1.81,18.8,22.5,69.4,47.3,14.1,56.8,60.2,99.27,51.1,6666
9,Phoenix Suns,67,37,30,3236,114.3,111.9,2.3,64.9,1.95,19.2,31.0,71.5,51.0,14.1,53.7,57.2,98.67,51.6,6649


### Cleaning Game Data

In [11]:
nba_team_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,-22.0
1,22022,1610612741,CHI,Chicago Bulls,0022201223,2023-04-09,CHI vs. DET,W,240,103,40,95,0.421,6,30,0.200,17,22,0.773,11,37,48,21,15,7,3,16,22.0
2,22022,1610612761,TOR,Toronto Raptors,0022201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0
3,22022,1610612737,ATL,Atlanta Hawks,0022201216,2023-04-09,ATL @ BOS,L,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,-6.0
4,22022,1610612745,HOU,Houston Rockets,0022201222,2023-04-09,HOU @ WAS,W,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22022,1610612765,DET,Detroit Pistons,0022200003,2022-10-19,DET vs. ORL,W,239,113,40,94,0.426,14,38,0.368,19,24,0.792,12,29,41,31,11,4,12,21,4.0
2456,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0
2457,22022,1610612747,LAL,Los Angeles Lakers,0022200002,2022-10-18,LAL @ GSW,L,241,109,40,94,0.426,10,40,0.250,19,25,0.760,9,39,48,23,12,4,21,18,-14.0
2458,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,45,99,0.455,16,45,0.356,17,23,0.739,11,37,48,31,11,4,18,23,14.0


In [12]:
nba_team_games[nba_team_games['GAME_ID']=='0022201228']

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
6,22022,1610612747,LAL,Los Angeles Lakers,0022201228,2023-04-09,LAL vs. UTA,W,240,128,47,97,0.485,18,45,0.400,16,21,0.762,10,39,49,28,4,7,10,9,11.0
15,22022,1610612762,UTA,Utah Jazz,0022201228,2023-04-09,UTA @ LAL,L,240,117,51,101,0.505,10,35,0.286,5,6,0.833,7,38,45,31,4,4,11,16,-11.0


In [13]:
nba_team_games.drop_duplicates('GAME_ID',keep='first', inplace=True)

In [14]:
nba_team_games[nba_team_games['GAME_ID']=='0022201228']

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
6,22022,1610612747,LAL,Los Angeles Lakers,0022201228,2023-04-09,LAL vs. UTA,W,240,128,47,97,0.485,18,45,0.4,16,21,0.762,10,39,49,28,4,7,10,9,11.0


In [15]:
nba_team_games['Opponent'] = nba_team_games['MATCHUP'].str.strip().str[-3:]

In [16]:
nba_team_games['Home'] = np.where(nba_team_games['MATCHUP'].str.contains('vs.'),1,0)

In [17]:
nba_team_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Opponent,Home
0,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,-22.0,CHI,0
2,22022,1610612761,TOR,Toronto Raptors,0022201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,MIL,1
3,22022,1610612737,ATL,Atlanta Hawks,0022201216,2023-04-09,ATL @ BOS,L,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,-6.0,BOS,0
4,22022,1610612745,HOU,Houston Rockets,0022201222,2023-04-09,HOU @ WAS,W,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,5.0,WAS,0
5,22022,1610612759,SAS,San Antonio Spurs,0022201224,2023-04-09,SAS @ DAL,W,240,138,51,104,0.490,15,38,0.395,21,30,0.700,13,56,69,31,2,3,12,13,21.0,DAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,22022,1610612753,ORL,Orlando Magic,0022200003,2022-10-19,ORL @ DET,L,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,-4.0,DET,0
2447,22022,1610612764,WAS,Washington Wizards,0022200004,2022-10-19,WAS @ IND,W,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,7.0,IND,0
2452,22022,1610612737,ATL,Atlanta Hawks,0022200005,2022-10-19,ATL vs. HOU,W,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,10.0,HOU,1
2456,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0,BOS,0


In [18]:
from datetime import date, timedelta

In [19]:
date_to_use = date.today() - timedelta(days=1)
date_to_use

datetime.date(2023, 4, 23)

In [20]:
nba_team_games = nba_team_games[nba_team_games['GAME_DATE']<pd.to_datetime(date_to_use)]

In [21]:
nba_team_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Opponent,Home
0,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,-22.0,CHI,0
2,22022,1610612761,TOR,Toronto Raptors,0022201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,MIL,1
3,22022,1610612737,ATL,Atlanta Hawks,0022201216,2023-04-09,ATL @ BOS,L,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,-6.0,BOS,0
4,22022,1610612745,HOU,Houston Rockets,0022201222,2023-04-09,HOU @ WAS,W,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,5.0,WAS,0
5,22022,1610612759,SAS,San Antonio Spurs,0022201224,2023-04-09,SAS @ DAL,W,240,138,51,104,0.490,15,38,0.395,21,30,0.700,13,56,69,31,2,3,12,13,21.0,DAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,22022,1610612753,ORL,Orlando Magic,0022200003,2022-10-19,ORL @ DET,L,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,-4.0,DET,0
2447,22022,1610612764,WAS,Washington Wizards,0022200004,2022-10-19,WAS @ IND,W,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,7.0,IND,0
2452,22022,1610612737,ATL,Atlanta Hawks,0022200005,2022-10-19,ATL vs. HOU,W,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,10.0,HOU,1
2456,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0,BOS,0


In [22]:
del nba_team_games['TEAM_NAME']
del nba_team_games['MATCHUP']
del nba_team_games['GAME_ID']
del nba_team_games['GAME_DATE']
del nba_team_games['SEASON_ID']
del nba_team_games['TEAM_ID']

In [23]:
nba_team_games

,TEAM_ABBREVIATION,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Opponent,Home
0,DET,L,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,-22.0,CHI,0
2,TOR,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,MIL,1
3,ATL,L,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,-6.0,BOS,0
4,HOU,W,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,5.0,WAS,0
5,SAS,W,240,138,51,104,0.490,15,38,0.395,21,30,0.700,13,56,69,31,2,3,12,13,21.0,DAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,ORL,L,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,-4.0,DET,0
2447,WAS,W,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,7.0,IND,0
2452,ATL,W,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,10.0,HOU,1
2456,PHI,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0,BOS,0


### EDA

In [24]:
home_df = nba_team_games[nba_team_games['Home']==1]
away_df = nba_team_games[nba_team_games['Home']==0]

In [25]:
len(home_df[home_df['WL']=='W']['WL'])/len(home_df['WL']), len(away_df[away_df['WL']=='W']['WL'])/len(away_df['WL'])

(0.5848142164781907, 0.4238952536824877)

### Training Models

In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [27]:
le = LabelEncoder()

In [28]:
nba_team_games.columns

Index(['TEAM_ABBREVIATION', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'Opponent', 'Home'],
      dtype='object')

In [29]:
nba_team_games_encoded = nba_team_games[['TEAM_ABBREVIATION', 'WL','Opponent']].apply(LabelEncoder().fit_transform)

In [30]:
old_nba_team_games = nba_team_games[['MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF','Home']]
new_nba_team_games = nba_team_games_encoded[['TEAM_ABBREVIATION', 'WL','Opponent']]

nba_team_games_final = pd.concat([old_nba_team_games,new_nba_team_games],axis=1)

In [31]:
nba_team_games_final

,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,Home,TEAM_ABBREVIATION,WL,Opponent
0,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,0,8,0,4
2,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,1,27,1,16
3,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,0,0,0,2
4,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,0,10,1,29
5,240,138,51,104,0.490,15,38,0.395,21,30,0.700,13,56,69,31,2,3,12,13,0,26,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,0,21,0,8
2447,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,0,29,1,11
2452,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,1,0,1,10
2456,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,0,22,0,2


In [32]:
X_data = nba_team_games_final.drop(['WL'],axis=1)
y_data = nba_team_games_final[['WL']]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

Random Forest Classifier

In [34]:
rfc = RandomForestClassifier().fit(X_train,y_train)

<ipython-input-34-2ae091266cb1>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc = RandomForestClassifier().fit(X_train,y_train)


In [35]:
rfc.score(X_test,y_test)

0.8008130081300813

In [36]:
rfc.feature_importances_

array([0.01844152, 0.10914567, 0.05635996, 0.04016933, 0.10230817,
       0.04426795, 0.02762234, 0.07840179, 0.03329638, 0.03083204,
       0.03713413, 0.02411395, 0.08935146, 0.06210511, 0.03489781,
       0.0339319 , 0.03314277, 0.03177331, 0.04140679, 0.01111504,
       0.02993844, 0.03024416])

In [37]:
feature_import = pd.DataFrame(rfc.feature_importances_)
column_names = pd.DataFrame(X_data.columns)

In [38]:
importance_df = pd.concat([column_names,feature_import],axis=1)

In [39]:
importance_df.columns = ['Feature','Importance']

In [40]:
importance_df.sort_values(by='Importance',ascending=False)

,Feature,Importance
1,PTS,0.109146
4,FG_PCT,0.102308
12,DREB,0.089351
7,FG3_PCT,0.078402
13,REB,0.062105
2,FGM,0.056360
5,FG3M,0.044268
18,PF,0.041407
3,FGA,0.040169
10,FT_PCT,0.037134


XG Boost

In [41]:
import xgboost as xgb

In [42]:
xgb_c = xgb.XGBClassifier()

In [43]:
xgb_c.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [44]:
xgb_c.score(X_test,y_test)

0.7845528455284553

In [45]:
pred = xgb_c.predict(X_test)

In [46]:
xgb_feature_import = pd.DataFrame(xgb_c.feature_importances_)

In [47]:
xgb_importance_df = pd.concat([column_names,xgb_feature_import],axis=1)

In [48]:
xgb_importance_df.columns = ['Feature','Importance']

In [49]:
xgb_importance_df.sort_values(by='Importance',ascending=False)

,Feature,Importance
1,PTS,0.196066
12,DREB,0.097611
7,FG3_PCT,0.057756
4,FG_PCT,0.052329
13,REB,0.044464
15,STL,0.041653
3,FGA,0.041603
19,Home,0.041430
18,PF,0.041312
17,TOV,0.040534


### Testing New Method

In [50]:
nba_team_games

,TEAM_ABBREVIATION,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Opponent,Home
0,DET,L,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,-22.0,CHI,0
2,TOR,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,MIL,1
3,ATL,L,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,-6.0,BOS,0
4,HOU,W,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,5.0,WAS,0
5,SAS,W,240,138,51,104,0.490,15,38,0.395,21,30,0.700,13,56,69,31,2,3,12,13,21.0,DAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,ORL,L,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,-4.0,DET,0
2447,WAS,W,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,7.0,IND,0
2452,ATL,W,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,10.0,HOU,1
2456,PHI,L,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,-9.0,BOS,0


In [51]:
nba_team_games_final

,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,Home,TEAM_ABBREVIATION,WL,Opponent
0,240,81,31,85,0.365,9,33,0.273,10,13,0.769,15,44,59,20,1,3,24,16,0,8,0,4
2,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,1,27,1,16
3,241,114,44,97,0.454,11,39,0.282,15,19,0.789,16,34,50,27,11,5,9,12,0,0,0,2
4,241,114,44,90,0.489,6,21,0.286,20,28,0.714,13,38,51,24,8,6,15,15,0,10,1,29
5,240,138,51,104,0.490,15,38,0.395,21,30,0.700,13,56,69,31,2,3,12,13,0,26,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,240,109,42,86,0.488,11,30,0.367,14,19,0.737,10,38,48,21,5,5,18,24,0,21,0,8
2447,241,114,42,92,0.457,11,31,0.355,19,24,0.792,14,39,53,21,5,10,16,19,0,29,1,11
2452,240,117,45,90,0.500,7,25,0.280,20,24,0.833,4,34,38,30,12,5,9,18,1,0,1,10
2456,239,117,40,80,0.500,13,34,0.382,24,28,0.857,4,27,31,16,8,3,14,25,0,22,0,2


In [52]:
team_code = pd.concat([nba_team_games[['TEAM_ABBREVIATION']],nba_team_games_final[['TEAM_ABBREVIATION']]],axis=1)
team_code

,TEAM_ABBREVIATION,TEAM_ABBREVIATION
0,DET,8
2,TOR,27
3,ATL,0
4,HOU,10
5,SAS,26
...,...,...
2445,ORL,21
2447,WAS,29
2452,ATL,0
2456,PHI,22


In [53]:
team_code.columns = ['TEAM_NAME','TEAM_CODE']

In [54]:
team_code_unique = team_code.drop_duplicates()

In [55]:
team_code_unique.sort_values(by='TEAM_CODE',ascending=True)

,TEAM_NAME,TEAM_CODE
3,ATL,0
75,BKN,1
74,BOS,2
85,CHA,3
68,CHI,4
21,CLE,5
38,DAL,6
14,DEN,7
0,DET,8
10,GSW,9


In [56]:
X_data_2 = nba_team_games_final[['TEAM_ABBREVIATION','Opponent']]
y_data_2 = nba_team_games_final[['WL']]

In [57]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_data_2, y_data_2, test_size=0.2, random_state=42)

Random Forest Classifier

In [58]:
rfc = RandomForestClassifier().fit(X_train_2,y_train_2)

<ipython-input-58-5ae8ad5eded3>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc = RandomForestClassifier().fit(X_train_2,y_train_2)


In [59]:
rfc.score(X_test_2,y_test_2)

0.5040650406504065

In [60]:
test_data = pd.DataFrame([[8,16]],columns=[['TEAM_ABBREVIATION','Opponent']])

In [61]:
test_data

,TEAM_ABBREVIATION,Opponent
0,8,16


In [62]:
X_data = test_data[['TEAM_ABBREVIATION','Opponent']]

In [63]:
results = rfc.predict_proba(X_data)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [64]:
results

array([[0.25, 0.75]])

In [65]:
rfc.predict(X_data)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1])

In [66]:
rfc.classes_

array([0, 1])

### Raptors Testing

Getting Raptors 2022-23 regular season games

In [67]:
TOR_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=1610612761).get_data_frames()[0]

TOR_games[['SEASON_ID', 'TEAM_ID','GAME_ID','MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']] = TOR_games[['SEASON_ID', 'TEAM_ID','GAME_ID','MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']].apply(lambda col:pd.to_numeric(col,errors='coerce'))

TOR_games['GAME_DATE'] = pd.to_datetime(TOR_games['GAME_DATE'],format='%Y-%m-%d')

TOR_games_2022 = TOR_games[TOR_games['GAME_DATE'] >= '2022-10-18']

In [68]:
TOR_games_2022.sort_values(by='GAME_DATE',ascending=False)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,52022,1610612761,TOR,Toronto Raptors,52200111,2023-04-12,TOR vs. CHI,L,238,105,38,87,0.437,11,31,0.355,18,36,0.500,16,34,50,23,5,5,14,18,-4.0
1,22022,1610612761,TOR,Toronto Raptors,22201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0
2,22022,1610612761,TOR,Toronto Raptors,22201206,2023-04-07,TOR @ BOS,L,241,102,40,93,0.430,9,31,0.290,13,17,0.765,12,23,35,21,12,2,7,17,-19.0
3,22022,1610612761,TOR,Toronto Raptors,22201192,2023-04-05,TOR @ BOS,L,241,93,37,90,0.411,6,33,0.182,13,17,0.765,13,37,50,20,10,7,11,19,-4.0
4,22022,1610612761,TOR,Toronto Raptors,22201176,2023-04-04,TOR @ CHA,W,240,120,42,88,0.477,15,33,0.455,21,28,0.750,10,31,41,27,13,7,6,18,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,22022,1610612761,TOR,Toronto Raptors,22200059,2022-10-26,TOR vs. PHI,W,238,119,46,84,0.548,16,37,0.432,11,14,0.786,6,33,39,32,9,5,9,21,10.0
79,22022,1610612761,TOR,Toronto Raptors,22200045,2022-10-24,TOR @ MIA,W,242,98,34,86,0.395,14,32,0.438,16,20,0.800,10,42,52,22,6,6,12,22,8.0
80,22022,1610612761,TOR,Toronto Raptors,22200031,2022-10-22,TOR @ MIA,L,241,109,37,87,0.425,14,39,0.359,21,28,0.750,8,29,37,24,9,5,6,25,-3.0
81,22022,1610612761,TOR,Toronto Raptors,22200021,2022-10-21,TOR @ BKN,L,240,105,41,84,0.488,11,28,0.393,12,18,0.667,8,33,41,25,9,4,13,21,-4.0


Getting active Raptors players

In [69]:
raps_players = franchiseplayers.FranchisePlayers(team_id=1610612761,per_mode_detailed='PerGame',season_type_all_star='Regular Season').get_data_frames()[0]
active_raps = raps_players[raps_players['ACTIVE_WITH_TEAM']==1]
active_raps.reset_index(inplace=True, drop=True)

In [70]:
active_raptors = active_raps[active_raps['GP']>=100]
active_raptors

,LEAGUE_ID,TEAM_ID,TEAM,PERSON_ID,PLAYER,SEASON_TYPE,ACTIVE_WITH_TEAM,GP,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS
7,00,1610612761,Toronto Raptors,1629018,Gary Trent Jr.,Regular Season,1,153,6.261437,14.915032,0.420,2.738562,7.313725,0.374,2.424836,2.875816,0.843,0.405228,2.372549,2.777777,1.758169,1.797385,1.601307,0.915032,0.241830,17.686274
9,00,1610612761,Toronto Raptors,1630173,Precious Achiuwa,Regular Season,1,128,3.601562,7.867187,0.458,0.664062,2.062500,0.322,1.289062,1.992187,0.647,1.921875,4.335937,6.257812,1.031250,1.976562,0.531250,1.117187,0.554687,9.156250
10,00,1610612761,Toronto Raptors,1627751,Jakob Poeltl,Regular Season,1,162,2.901234,4.500000,0.645,0.006172,0.012345,0.500,0.814814,1.419753,0.574,2.018518,2.888888,4.907407,0.771604,2.481481,0.530864,0.882716,0.950617,6.623456
11,00,1610612761,Toronto Raptors,1627783,Pascal Siakam,Regular Season,1,471,6.450106,13.233545,0.487,1.021231,3.118895,0.327,3.084925,3.987261,0.774,1.454352,5.080679,6.535031,3.515923,2.787685,0.925690,1.872611,0.651804,17.006369
12,00,1610612761,Toronto Raptors,1627832,Fred VanVleet,Regular Season,1,417,4.923261,12.249400,0.402,2.398081,6.436450,0.373,2.359712,2.714628,0.869,0.417266,2.904076,3.321342,5.273381,2.115107,1.347721,1.671462,0.405275,14.604316
13,00,1610612761,Toronto Raptors,1628384,O.G. Anunoby,Regular Season,1,368,4.377717,9.285326,0.471,1.603260,4.271739,0.375,1.239130,1.665760,0.744,1.100543,3.187500,4.288043,1.521739,2.396739,1.247282,1.266304,0.510869,11.597826
14,00,1610612761,Toronto Raptors,1628449,Chris Boucher,Regular Season,1,306,3.241830,6.669934,0.486,0.882352,2.640522,0.334,1.745098,2.238562,0.780,1.882352,3.516339,5.398692,0.473856,2.052287,0.519607,0.535947,1.094771,9.111111
15,00,1610612761,Toronto Raptors,1630201,Malachi Flynn,Regular Season,1,144,1.986111,5.312500,0.374,0.909722,2.715277,0.335,0.590277,0.784722,0.752,0.243055,1.527777,1.770833,1.923611,1.236111,0.548611,0.562500,0.104166,5.472222
16,00,1610612761,Toronto Raptors,1630567,Scottie Barnes,Regular Season,1,151,6.105960,12.900662,0.473,0.801324,2.761589,0.290,2.304635,3.052980,0.755,2.476821,4.602649,7.079470,4.152317,2.397350,1.079470,1.920529,0.768211,15.317880


Creating combined df

In [71]:
trentjr_gamelog = playergamelog.PlayerGameLog(player_id=1629018,season_type_all_star='Regular Season').get_data_frames()[0]
achiuwa_gamelog = playergamelog.PlayerGameLog(player_id=1630173,season_type_all_star='Regular Season').get_data_frames()[0]
poeltl_gamelog = playergamelog.PlayerGameLog(player_id=1627751,season_type_all_star='Regular Season').get_data_frames()[0]
siakam_gamelog = playergamelog.PlayerGameLog(player_id=1627783,season_type_all_star='Regular Season').get_data_frames()[0]
vanvleet_gamelog = playergamelog.PlayerGameLog(player_id=1627832,season_type_all_star='Regular Season').get_data_frames()[0]
anunoby_gamelog = playergamelog.PlayerGameLog(player_id=1628384,season_type_all_star='Regular Season').get_data_frames()[0]
boucher_gamelog = playergamelog.PlayerGameLog(player_id=1628449,season_type_all_star='Regular Season').get_data_frames()[0]
flynn_gamelog = playergamelog.PlayerGameLog(player_id=1630201,season_type_all_star='Regular Season').get_data_frames()[0]
barnes_gamelog = playergamelog.PlayerGameLog(player_id=1630567,season_type_all_star='Regular Season').get_data_frames()[0]

In [72]:
trentjr_gamelog['GAME_DATE'] = pd.to_datetime(trentjr_gamelog['GAME_DATE'],format='%b %d, %Y')
achiuwa_gamelog['GAME_DATE'] = pd.to_datetime(achiuwa_gamelog['GAME_DATE'],format='%b %d, %Y')
poeltl_gamelog['GAME_DATE'] = pd.to_datetime(poeltl_gamelog['GAME_DATE'],format='%b %d, %Y')
siakam_gamelog['GAME_DATE'] = pd.to_datetime(siakam_gamelog['GAME_DATE'],format='%b %d, %Y')
vanvleet_gamelog['GAME_DATE'] = pd.to_datetime(vanvleet_gamelog['GAME_DATE'],format='%b %d, %Y')
anunoby_gamelog['GAME_DATE'] = pd.to_datetime(anunoby_gamelog['GAME_DATE'],format='%b %d, %Y')
boucher_gamelog['GAME_DATE'] = pd.to_datetime(boucher_gamelog['GAME_DATE'],format='%b %d, %Y')
flynn_gamelog['GAME_DATE'] = pd.to_datetime(flynn_gamelog['GAME_DATE'],format='%b %d, %Y')
barnes_gamelog['GAME_DATE'] = pd.to_datetime(barnes_gamelog['GAME_DATE'],format='%b %d, %Y')

In [73]:
vanvleet_gamelog

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,1627832,0022201206,2023-04-07,TOR @ BOS,L,31,3,11,0.273,2,6,0.333,4,4,1.000,0,0,0,8,2,0,2,3,12,-12,1
1,22022,1627832,0022201192,2023-04-05,TOR @ BOS,L,37,2,14,0.143,1,12,0.083,2,2,1.000,1,3,4,5,1,1,1,3,7,4,1
2,22022,1627832,0022201176,2023-04-04,TOR @ CHA,W,27,4,9,0.444,3,5,0.600,5,6,0.833,0,1,1,6,4,1,0,2,16,12,1
3,22022,1627832,0022201163,2023-04-02,TOR @ CHA,W,38,7,17,0.412,1,8,0.125,5,5,1.000,1,4,5,20,3,0,2,4,20,18,1
4,22022,1627832,0022201150,2023-03-31,TOR @ PHI,L,42,5,17,0.294,3,10,0.300,6,7,0.857,0,5,5,6,1,0,6,5,19,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,22022,1627832,0022200059,2022-10-26,TOR vs. PHI,W,39,5,12,0.417,2,5,0.400,3,3,1.000,0,6,6,8,2,3,2,1,15,8,1
65,22022,1627832,0022200045,2022-10-24,TOR @ MIA,W,39,6,16,0.375,4,6,0.667,8,9,0.889,2,2,4,9,1,0,2,1,24,13,1
66,22022,1627832,0022200031,2022-10-22,TOR @ MIA,L,39,2,9,0.222,1,6,0.167,2,2,1.000,1,1,2,10,3,0,1,5,7,6,1
67,22022,1627832,0022200021,2022-10-21,TOR @ BKN,L,38,7,11,0.636,4,6,0.667,0,0,0.000,1,6,7,9,4,1,0,3,18,3,1


In [74]:
TOR_games_2022

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,52022,1610612761,TOR,Toronto Raptors,52200111,2023-04-12,TOR vs. CHI,L,238,105,38,87,0.437,11,31,0.355,18,36,0.500,16,34,50,23,5,5,14,18,-4.0
1,22022,1610612761,TOR,Toronto Raptors,22201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0
2,22022,1610612761,TOR,Toronto Raptors,22201206,2023-04-07,TOR @ BOS,L,241,102,40,93,0.430,9,31,0.290,13,17,0.765,12,23,35,21,12,2,7,17,-19.0
3,22022,1610612761,TOR,Toronto Raptors,22201192,2023-04-05,TOR @ BOS,L,241,93,37,90,0.411,6,33,0.182,13,17,0.765,13,37,50,20,10,7,11,19,-4.0
4,22022,1610612761,TOR,Toronto Raptors,22201176,2023-04-04,TOR @ CHA,W,240,120,42,88,0.477,15,33,0.455,21,28,0.750,10,31,41,27,13,7,6,18,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,22022,1610612761,TOR,Toronto Raptors,22200059,2022-10-26,TOR vs. PHI,W,238,119,46,84,0.548,16,37,0.432,11,14,0.786,6,33,39,32,9,5,9,21,10.0
79,22022,1610612761,TOR,Toronto Raptors,22200045,2022-10-24,TOR @ MIA,W,242,98,34,86,0.395,14,32,0.438,16,20,0.800,10,42,52,22,6,6,12,22,8.0
80,22022,1610612761,TOR,Toronto Raptors,22200031,2022-10-22,TOR @ MIA,L,241,109,37,87,0.425,14,39,0.359,21,28,0.750,8,29,37,24,9,5,6,25,-3.0
81,22022,1610612761,TOR,Toronto Raptors,22200021,2022-10-21,TOR @ BKN,L,240,105,41,84,0.488,11,28,0.393,12,18,0.667,8,33,41,25,9,4,13,21,-4.0


In [75]:
TOR_games_2022['GAME_DATE'][0]

Timestamp('2023-04-12 00:00:00')

In [76]:
trentjr_gamelog

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,1629018,0022201221,2023-04-09,TOR vs. MIL,W,27,7,13,0.538,4,7,0.571,5,5,1.000,0,1,1,3,0,0,0,0,23,14,1
1,22022,1629018,0022201206,2023-04-07,TOR @ BOS,L,21,4,10,0.400,0,4,0.000,1,2,0.500,1,2,3,3,1,0,1,1,9,-13,1
2,22022,1629018,0022201192,2023-04-05,TOR @ BOS,L,16,0,5,0.000,0,2,0.000,0,0,0.000,0,2,2,0,0,0,1,1,0,-5,1
3,22022,1629018,0022201068,2023-03-19,TOR @ MIL,L,33,6,18,0.333,5,10,0.500,1,2,0.500,1,2,3,0,1,0,0,0,18,3,1
4,22022,1629018,0022201058,2023-03-18,TOR vs. MIN,W,29,8,14,0.571,2,5,0.400,1,1,1.000,1,2,3,3,5,0,1,1,19,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,22022,1629018,0022200059,2022-10-26,TOR vs. PHI,W,35,11,16,0.688,5,10,0.500,0,0,0.000,0,0,0,0,2,0,2,1,27,2,1
62,22022,1629018,0022200045,2022-10-24,TOR @ MIA,W,42,7,19,0.368,4,12,0.333,0,0,0.000,1,0,1,2,1,0,1,1,18,5,1
63,22022,1629018,0022200031,2022-10-22,TOR @ MIA,L,39,7,18,0.389,5,12,0.417,4,6,0.667,0,1,1,2,2,1,1,3,23,-6,1
64,22022,1629018,0022200021,2022-10-21,TOR @ BKN,L,36,6,16,0.375,2,7,0.286,0,0,0.000,0,1,1,0,1,0,0,0,14,-9,1


In [77]:
TOR_games_2022['Trent Jr.'] = 0
TOR_games_2022['Achiuwa'] = 0
TOR_games_2022['Poeltl'] = 0
TOR_games_2022['Siakam'] = 0
TOR_games_2022['VanVleet'] = 0
TOR_games_2022['Anunoby'] = 0
TOR_games_2022['Boucher'] = 0
TOR_games_2022['Flynn'] = 0
TOR_games_2022['Barnes'] = 0

<ipython-input-77-0b542c804708>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022['Trent Jr.'] = 0
<ipython-input-77-0b542c804708>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022['Achiuwa'] = 0
<ipython-input-77-0b542c804708>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [78]:
TOR_games_2022.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Trent Jr.,Achiuwa,Poeltl,Siakam,VanVleet,Anunoby,Boucher,Flynn,Barnes
0,52022,1610612761,TOR,Toronto Raptors,52200111,2023-04-12,TOR vs. CHI,L,238,105,38,87,0.437,11,31,0.355,18,36,0.500,16,34,50,23,5,5,14,18,-4.0,0,0,0,0,0,0,0,0,0
1,22022,1610612761,TOR,Toronto Raptors,22201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,0,0,0,0,0,0,0,0,0
2,22022,1610612761,TOR,Toronto Raptors,22201206,2023-04-07,TOR @ BOS,L,241,102,40,93,0.430,9,31,0.290,13,17,0.765,12,23,35,21,12,2,7,17,-19.0,0,0,0,0,0,0,0,0,0
3,22022,1610612761,TOR,Toronto Raptors,22201192,2023-04-05,TOR @ BOS,L,241,93,37,90,0.411,6,33,0.182,13,17,0.765,13,37,50,20,10,7,11,19,-4.0,0,0,0,0,0,0,0,0,0
4,22022,1610612761,TOR,Toronto Raptors,22201176,2023-04-04,TOR @ CHA,W,240,120,42,88,0.477,15,33,0.455,21,28,0.750,10,31,41,27,13,7,6,18,20.0,0,0,0,0,0,0,0,0,0


In [79]:
def game_played_checker(df,name):
  for i in range(0,len(TOR_games_2022['GAME_DATE'])):
    date = TOR_games_2022['GAME_DATE'][i]
    if any(df[df['GAME_DATE']==date]['MIN']):
      TOR_games_2022[name][i] = 1
    else:
      TOR_games_2022[name][i] = 0

In [80]:
game_played_checker(trentjr_gamelog,'Trent Jr.')
game_played_checker(achiuwa_gamelog,'Achiuwa')
game_played_checker(poeltl_gamelog,'Poeltl')
game_played_checker(siakam_gamelog,'Siakam')
game_played_checker(vanvleet_gamelog,'VanVleet')
game_played_checker(anunoby_gamelog,'Anunoby')
game_played_checker(boucher_gamelog,'Boucher')
game_played_checker(flynn_gamelog,'Flynn')
game_played_checker(barnes_gamelog,'Barnes')

<ipython-input-79-8da59d2130fc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022[name][i] = 0
<ipython-input-79-8da59d2130fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022[name][i] = 1


In [81]:
TOR_games_2022.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Trent Jr.,Achiuwa,Poeltl,Siakam,VanVleet,Anunoby,Boucher,Flynn,Barnes
0,52022,1610612761,TOR,Toronto Raptors,52200111,2023-04-12,TOR vs. CHI,L,238,105,38,87,0.437,11,31,0.355,18,36,0.500,16,34,50,23,5,5,14,18,-4.0,0,0,0,0,0,0,0,0,0
1,22022,1610612761,TOR,Toronto Raptors,22201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,1,1,1,0,0,0,1,1,1
2,22022,1610612761,TOR,Toronto Raptors,22201206,2023-04-07,TOR @ BOS,L,241,102,40,93,0.430,9,31,0.290,13,17,0.765,12,23,35,21,12,2,7,17,-19.0,1,1,1,1,1,1,1,1,1
3,22022,1610612761,TOR,Toronto Raptors,22201192,2023-04-05,TOR @ BOS,L,241,93,37,90,0.411,6,33,0.182,13,17,0.765,13,37,50,20,10,7,11,19,-4.0,1,1,1,1,1,1,1,0,1
4,22022,1610612761,TOR,Toronto Raptors,22201176,2023-04-04,TOR @ CHA,W,240,120,42,88,0.477,15,33,0.455,21,28,0.750,10,31,41,27,13,7,6,18,20.0,0,1,1,1,1,1,1,1,1


In [82]:
TOR_games_2022['Opponent'] = TOR_games_2022['MATCHUP'].str.strip().str[-3:]

<ipython-input-82-7875bdf98535>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022['Opponent'] = TOR_games_2022['MATCHUP'].str.strip().str[-3:]


In [83]:
TOR_games_2022['Home'] = np.where(TOR_games_2022['MATCHUP'].str.contains('vs.'),1,0)

<ipython-input-83-9228ed91a27d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022['Home'] = np.where(TOR_games_2022['MATCHUP'].str.contains('vs.'),1,0)


In [84]:
vv_cond = TOR_games_2022['VanVleet']==1
siakam_cond = TOR_games_2022['Siakam']==1
barnes_cond = TOR_games_2022['Barnes']==1
anunoby_cond = TOR_games_2022['Anunoby']==1
poeltl_cond = TOR_games_2022['Poeltl']==1

TOR_games_2022['Starters_In'] = np.where(vv_cond & siakam_cond & barnes_cond & anunoby_cond & poeltl_cond,1,0)

<ipython-input-84-285265f4169b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOR_games_2022['Starters_In'] = np.where(vv_cond & siakam_cond & barnes_cond & anunoby_cond & poeltl_cond,1,0)


In [85]:
TOR_games_2022.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Trent Jr.,Achiuwa,Poeltl,Siakam,VanVleet,Anunoby,Boucher,Flynn,Barnes,Opponent,Home,Starters_In
0,52022,1610612761,TOR,Toronto Raptors,52200111,2023-04-12,TOR vs. CHI,L,238,105,38,87,0.437,11,31,0.355,18,36,0.500,16,34,50,23,5,5,14,18,-4.0,0,0,0,0,0,0,0,0,0,CHI,1,0
1,22022,1610612761,TOR,Toronto Raptors,22201221,2023-04-09,TOR vs. MIL,W,239,121,48,95,0.505,11,33,0.333,14,16,0.875,7,38,45,29,8,2,9,19,16.0,1,1,1,0,0,0,1,1,1,MIL,1,0
2,22022,1610612761,TOR,Toronto Raptors,22201206,2023-04-07,TOR @ BOS,L,241,102,40,93,0.430,9,31,0.290,13,17,0.765,12,23,35,21,12,2,7,17,-19.0,1,1,1,1,1,1,1,1,1,BOS,0,1
3,22022,1610612761,TOR,Toronto Raptors,22201192,2023-04-05,TOR @ BOS,L,241,93,37,90,0.411,6,33,0.182,13,17,0.765,13,37,50,20,10,7,11,19,-4.0,1,1,1,1,1,1,1,0,1,BOS,0,1
4,22022,1610612761,TOR,Toronto Raptors,22201176,2023-04-04,TOR @ CHA,W,240,120,42,88,0.477,15,33,0.455,21,28,0.750,10,31,41,27,13,7,6,18,20.0,0,1,1,1,1,1,1,1,1,CHA,0,1


In [86]:
TOR_games_2022_ML = TOR_games_2022[['TEAM_ABBREVIATION','Opponent','Home','Starters_In','Trent Jr.','Achiuwa','Poeltl','Siakam','VanVleet','Anunoby','Boucher','Flynn','Barnes','WL']]

In [87]:
TOR_games_2022_ML.head()

,TEAM_ABBREVIATION,Opponent,Home,Starters_In,Trent Jr.,Achiuwa,Poeltl,Siakam,VanVleet,Anunoby,Boucher,Flynn,Barnes,WL
0,TOR,CHI,1,0,0,0,0,0,0,0,0,0,0,L
1,TOR,MIL,1,0,1,1,1,0,0,0,1,1,1,W
2,TOR,BOS,0,1,1,1,1,1,1,1,1,1,1,L
3,TOR,BOS,0,1,1,1,1,1,1,1,1,0,1,L
4,TOR,CHA,0,1,0,1,1,1,1,1,1,1,1,W


In [88]:
TOR_games_2022_ML_encoded = TOR_games_2022_ML[['TEAM_ABBREVIATION', 'Opponent','WL']].apply(LabelEncoder().fit_transform)

In [89]:
old_TOR_games = TOR_games_2022_ML[['Home','Starters_In','Trent Jr.','Achiuwa','Poeltl','Siakam','VanVleet','Anunoby','Boucher','Flynn','Barnes']]
new_TOR_games = TOR_games_2022_ML_encoded[['Opponent','WL']]

TOR_games_final = pd.concat([old_TOR_games,new_TOR_games],axis=1)

In [90]:
TOR_games_final

,Home,Starters_In,Trent Jr.,Achiuwa,Poeltl,Siakam,VanVleet,Anunoby,Boucher,Flynn,Barnes,Opponent,WL
0,1,0,0,0,0,0,0,0,0,0,0,4,0
1,1,0,1,1,1,0,0,0,1,1,1,16,1
2,0,1,1,1,1,1,1,1,1,1,1,2,0
3,0,1,1,1,1,1,1,1,1,0,1,2,0
4,0,1,0,1,1,1,1,1,1,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1,1,1,1,1,1,1,1,1,0,1,22,1
79,0,0,1,1,1,1,1,1,1,1,0,15,1
80,0,1,1,1,1,1,1,1,0,1,1,15,0
81,0,1,1,1,1,1,1,1,0,0,1,1,0


In [91]:
X_data_tor = TOR_games_final[['Home','Starters_In','Trent Jr.','Achiuwa','Poeltl','Siakam','VanVleet','Anunoby','Boucher','Flynn','Barnes','Opponent']]
y_data_tor = TOR_games_final[['WL']]

In [92]:
X_train_tor, X_test_tor, y_train_tor, y_test_tor = train_test_split(X_data_tor, y_data_tor, test_size=0.2, random_state=6)

Random Forest Classifier

In [93]:
rfc = RandomForestClassifier().fit(X_train_tor,y_train_tor)

<ipython-input-93-048d91b73603>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc = RandomForestClassifier().fit(X_train_tor,y_train_tor)


In [94]:
rfc.score(X_test_tor,y_test_tor)

0.47058823529411764

In [95]:
rfc.feature_importances_

array([0.13327367, 0.03685236, 0.05572807, 0.07388724, 0.05479732,
       0.02854529, 0.04480366, 0.04002349, 0.02404106, 0.06758925,
       0.02265936, 0.41779921])

### OKC Game Testing

Setting up initial df

In [96]:
teams.get_teams()

[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Georgia',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970},
 {'id': 1610612740,
  'full_name': 'New Orleans Pelicans',
  'abbreviation': 'NOP',
  'nickname': 'Pelicans',
  'city': 'New Orleans',
  'state': 'Louisiana',
  'year_founded': 2002},
 {'id': 1610612741,
  'full_name': 'Chicago Bulls',
  'abbreviation': 'CHI',
  'nickname': 'Bulls',
  'city': 'Chicago',
  'state': 'Illinois',
  'year_founded': 1966},
 {'id': 1610612742,
  'full_name': 'Dallas Mavericks',
  'abbreviation': 'DAL',
  'nickname': 'Mavericks',
  'city': 'Dallas',

In [97]:
OKC_games = leaguegamefinder.LeagueGameFinder(team_id_nullable=1610612760).get_data_frames()[0]

OKC_games[['SEASON_ID', 'TEAM_ID','GAME_ID','MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']] = OKC_games[['SEASON_ID', 'TEAM_ID','GAME_ID','MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']].apply(lambda col:pd.to_numeric(col,errors='coerce'))

OKC_games['GAME_DATE'] = pd.to_datetime(OKC_games['GAME_DATE'],format='%Y-%m-%d')

OKC_games_2022 = OKC_games[OKC_games['GAME_DATE'] >= '2022-10-18']

In [98]:
okc_players = franchiseplayers.FranchisePlayers(team_id=1610612760,per_mode_detailed='PerGame',season_type_all_star='Regular Season').get_data_frames()[0]
active_okc = okc_players[okc_players['ACTIVE_WITH_TEAM']==1]
active_okc.reset_index(inplace=True, drop=True)

In [99]:
active_okc_100 = active_okc[active_okc['GP']>=100]

In [100]:
active_okc_100

,LEAGUE_ID,TEAM_ID,TEAM,PERSON_ID,PLAYER,SEASON_TYPE,ACTIVE_WITH_TEAM,GP,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS
5,00,1610612760,Oklahoma City Thunder,1629026,Kenrich Williams,Regular Season,1,168,3.238095,6.392857,0.507,0.851190,2.214285,0.384,0.494047,0.934523,0.529,1.523809,2.964285,4.488095,2.172619,2.000000,0.857142,0.916666,0.244047,7.821428
7,00,1610612760,Oklahoma City Thunder,1628983,Shai Gilgeous-Alexander,Regular Season,1,229,8.497816,17.524017,0.485,1.331877,3.868995,0.344,6.419213,7.554585,0.850,0.707423,4.467248,5.174672,4.982532,2.275109,1.262008,2.558951,0.790393,24.746724
8,00,1610612760,Oklahoma City Thunder,1629652,Luguentz Dort,Regular Season,1,213,4.530516,11.530516,0.393,1.915492,5.774647,0.332,2.464788,3.131455,0.787,1.070422,2.807511,3.877934,1.676056,2.924882,0.924882,1.319248,0.300469,13.441314
9,00,1610612760,Oklahoma City Thunder,1630197,Aleksej Pokusevski,Regular Season,1,140,3.092857,7.950000,0.389,1.092857,3.628571,0.301,0.628571,0.907142,0.693,1.057142,3.864285,4.921428,2.085714,1.457142,0.564285,1.685714,0.878571,7.907142
11,00,1610612760,Oklahoma City Thunder,1630544,Tre Mann,Regular Season,1,127,3.370078,8.574803,0.393,1.409448,4.165354,0.338,0.842519,1.078740,0.781,0.409448,2.157480,2.566929,1.661417,1.582677,0.685039,1.047244,0.181102,8.992125
12,00,1610612760,Oklahoma City Thunder,1630581,Josh Giddey,Regular Season,1,130,6.307692,13.753846,0.459,1.015384,3.438461,0.295,1.246153,1.723076,0.723,1.892307,5.953846,7.846153,6.261538,1.776923,0.830769,2.938461,0.392307,14.876923
13,00,1610612760,Oklahoma City Thunder,1630598,Aaron Wiggins,Regular Season,1,120,2.850000,5.833333,0.489,0.750000,2.166666,0.346,1.008333,1.300000,0.776,0.991666,2.241666,3.233333,1.233333,1.691666,0.575000,0.908333,0.208333,7.458333


In [101]:
OKC_games_2022['Opponent'] = OKC_games_2022['MATCHUP'].str.strip().str[-3:]
OKC_games_2022['Home'] = np.where(OKC_games_2022['MATCHUP'].str.contains('vs.'),1,0)

<ipython-input-101-fb7a1e92cbc0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OKC_games_2022['Opponent'] = OKC_games_2022['MATCHUP'].str.strip().str[-3:]
<ipython-input-101-fb7a1e92cbc0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OKC_games_2022['Home'] = np.where(OKC_games_2022['MATCHUP'].str.contains('vs.'),1,0)


Getting player gamelogs

In [102]:
kwilliams_gamelog = playergamelog.PlayerGameLog(player_id=1629026,season_type_all_star='Regular Season').get_data_frames()[0]
sga_gamelog = playergamelog.PlayerGameLog(player_id=1628983,season_type_all_star='Regular Season').get_data_frames()[0]
ludort_gamelog = playergamelog.PlayerGameLog(player_id=1629652,season_type_all_star='Regular Season').get_data_frames()[0]
ap_gamelog = playergamelog.PlayerGameLog(player_id=1630197,season_type_all_star='Regular Season').get_data_frames()[0]
mann_gamelog = playergamelog.PlayerGameLog(player_id=1630544,season_type_all_star='Regular Season').get_data_frames()[0]
giddey_gamelog = playergamelog.PlayerGameLog(player_id=1630581,season_type_all_star='Regular Season').get_data_frames()[0]
wiggins_gamelog = playergamelog.PlayerGameLog(player_id=1630598,season_type_all_star='Regular Season').get_data_frames()[0]

In [103]:
kwilliams_gamelog['GAME_DATE'] = pd.to_datetime(kwilliams_gamelog['GAME_DATE'],format='%b %d, %Y')
sga_gamelog['GAME_DATE'] = pd.to_datetime(sga_gamelog['GAME_DATE'],format='%b %d, %Y')
ludort_gamelog['GAME_DATE'] = pd.to_datetime(ludort_gamelog['GAME_DATE'],format='%b %d, %Y')
ap_gamelog['GAME_DATE'] = pd.to_datetime(ap_gamelog['GAME_DATE'],format='%b %d, %Y')
mann_gamelog['GAME_DATE'] = pd.to_datetime(mann_gamelog['GAME_DATE'],format='%b %d, %Y')
giddey_gamelog['GAME_DATE'] = pd.to_datetime(giddey_gamelog['GAME_DATE'],format='%b %d, %Y')
wiggins_gamelog['GAME_DATE'] = pd.to_datetime(wiggins_gamelog['GAME_DATE'],format='%b %d, %Y')

In [104]:
def game_played_checker_OKC(df,name):
  for i in range(0,len(OKC_games_2022['GAME_DATE'])):
    date = OKC_games_2022['GAME_DATE'][i]
    if any(df[df['GAME_DATE']==date]['MIN']):
      OKC_games_2022[name][i] = 1
    else:
      OKC_games_2022[name][i] = 0

In [105]:
OKC_games_2022['Williams'] = 0
OKC_games_2022['Gilgeous-Alexander'] = 0
OKC_games_2022['Dort'] = 0
OKC_games_2022['Pokusevski'] = 0
OKC_games_2022['Mann'] = 0
OKC_games_2022['Giddey'] = 0
OKC_games_2022['Wiggins'] = 0

<ipython-input-105-f768844e9864>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OKC_games_2022['Williams'] = 0
<ipython-input-105-f768844e9864>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OKC_games_2022['Gilgeous-Alexander'] = 0
<ipython-input-105-f768844e9864>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [106]:
game_played_checker_OKC(kwilliams_gamelog,'Williams')
game_played_checker_OKC(sga_gamelog,'Gilgeous-Alexander')
game_played_checker_OKC(ludort_gamelog,'Dort')
game_played_checker_OKC(ap_gamelog,'Pokusevski')
game_played_checker_OKC(mann_gamelog,'Mann')
game_played_checker_OKC(giddey_gamelog,'Giddey')
game_played_checker_OKC(wiggins_gamelog,'Wiggins')

<ipython-input-104-321055282c38>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OKC_games_2022[name][i] = 0
<ipython-input-104-321055282c38>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OKC_games_2022[name][i] = 1


In [107]:
OKC_games_2022.head(5)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,Opponent,Home,Williams,Gilgeous-Alexander,Dort,Pokusevski,Mann,Giddey,Wiggins
0,52022,1610612760,OKC,Oklahoma City Thunder,52200211,2023-04-14,OKC @ MIN,L,239,95,32,89,0.360,13,40,0.325,18,20,0.900,12,30,42,19,9,6,17,20,-25.0,MIN,0,0,0,0,0,0,0,0
1,52022,1610612760,OKC,Oklahoma City Thunder,52200131,2023-04-12,OKC @ NOP,W,239,123,42,92,0.457,11,37,0.297,28,32,0.875,13,25,38,25,7,5,7,19,5.4,NOP,0,0,0,0,0,0,0,0
2,22022,1610612760,OKC,Oklahoma City Thunder,22201226,2023-04-09,OKC vs. MEM,W,240,115,45,90,0.500,12,36,0.333,13,16,0.813,8,41,49,30,5,2,14,17,15.0,MEM,1,0,0,0,0,1,0,1
3,22022,1610612760,OKC,Oklahoma City Thunder,22201200,2023-04-06,OKC @ UTA,W,242,114,39,97,0.402,15,45,0.333,21,26,0.808,17,33,50,29,12,2,12,17,16.0,UTA,0,0,1,1,0,1,1,1
4,22022,1610612760,OKC,Oklahoma City Thunder,22201187,2023-04-04,OKC @ GSW,L,241,125,44,93,0.473,11,32,0.344,26,35,0.743,13,25,38,23,4,1,10,22,-11.0,GSW,0,0,1,1,0,1,1,1


In [108]:
OKC_games_2022_ML = OKC_games_2022[['TEAM_ABBREVIATION','Opponent','Home','Williams','Gilgeous-Alexander','Dort','Pokusevski','Mann','Giddey','Wiggins','WL']]

In [109]:
OKC_games_2022_ML_encoded = OKC_games_2022_ML[['TEAM_ABBREVIATION', 'Opponent','WL']].apply(LabelEncoder().fit_transform)

In [110]:
old_OKC_games = OKC_games_2022_ML[['Home','Williams','Gilgeous-Alexander','Dort','Pokusevski','Mann','Giddey','Wiggins']]
new_OKC_games = OKC_games_2022_ML_encoded[['Opponent','WL']]

OKC_games_final = pd.concat([old_OKC_games,new_OKC_games],axis=1)

In [111]:
OKC_games_final

,Home,Williams,Gilgeous-Alexander,Dort,Pokusevski,Mann,Giddey,Wiggins,Opponent,WL
0,0,0,0,0,0,0,0,0,17,0
1,0,0,0,0,0,0,0,0,18,1
2,1,0,0,0,0,1,0,1,14,1
3,0,0,1,1,0,1,1,1,27,1
4,0,0,1,1,0,1,1,1,9,0
...,...,...,...,...,...,...,...,...,...,...
79,1,1,1,1,1,1,0,1,12,1
80,1,1,1,1,1,1,0,1,12,1
81,1,1,0,1,1,1,1,1,17,0
82,0,1,1,1,1,1,1,1,7,0


In [112]:
X_data_okc = OKC_games_final[['Home','Williams','Gilgeous-Alexander','Dort','Pokusevski','Mann','Giddey','Wiggins','Opponent']]
y_data_okc = OKC_games_final[['WL']]

In [113]:
X_train_okc, X_test_okc, y_train_okc, y_test_okc = train_test_split(X_data_okc, y_data_okc, test_size=0.2, random_state=6)

Random Forest Classifier

In [114]:
rfc = RandomForestClassifier().fit(X_train_okc,y_train_okc)

<ipython-input-114-67b3b69957ed>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc = RandomForestClassifier().fit(X_train_okc,y_train_okc)


In [115]:
rfc.score(X_test_okc,y_test_okc)

0.5882352941176471

In [116]:
rfc.feature_importances_

array([0.06499402, 0.05663748, 0.03709157, 0.02904864, 0.07074701,
       0.04631441, 0.09624004, 0.04653469, 0.55239214])

In both the Raptors and OKC explorations, the most important feature in determining whether the team would win or lose was the opponent being played.